<span style="font-size: 20pt; font-weight: bold; color: #0098cd;">Implementación del algortimo CKY</span>
 
 * Samuel Méndez Villegas - A01652277 
 * Mariana Pérez Carmona - A01731813
 * Paul Martin Garcia Morfin - A01750164
 * Nancy Lesly Segura - A01734337
 * Iker Ledesma Durán - A01653115

In [1]:
## Librería utilizada para realizar visualización del resultado
from anytree import Node, RenderTree

* Paso 1. Implementar en Python la representación de la gramática proporcionada.

In [2]:
R = {'0': ['S', ['NP', 'VP'], 0.800],
     '1': ['NP', ['time', 'flies', 'arrow'], 0.002],
     '2': ['NP', ['Det', 'Nominal'], 0.300],
     '3': ['NP', ['Nominal', 'Nominal'], 0.200],
     '4': ['Nominal', ['time', 'flies', 'arrow'], 0.002],
     '5': ['Nominal', ['Nominal', 'Noun'], 0.100],
     '6': ['Nominal', ['Nominal', 'PP'], 0.200],
     '7': ['VP', ['time'], 0.004],
     '8': ['VP', ['flies', 'like'], 0.008],
     '9': ['VP', ['Verb', 'NP'], 0.300],
     '10': ['VP', ['Verb', 'PP'], 0.200],
     '11': ['PP', ['Preposition', 'NP'], 0.100],
     '12': ['Verb', ['time'], 0.010],
     '13': ['Verb', ['flies', 'like'], 0.020],
     '14': ['Noun', ['time', 'flies', 'arrow'], 0.010],
     '15': ['Det', ['an'],0.050],
     '16': ['Preposition', ['like'], 0.050]}

In [3]:
phrase = 'time flies like an arrow'

* Paso 2. Implementar en Python la estructura de la matriz de análisis sintáctico (CKY).

In [4]:
## CREACIÓN DE LA MATRIZ

def create_matrix(phrase):
    '''
    Crea una matriz nxn en donde n es el número de palabras que conforman la frase "phrase"
    
    INPUT
    phrase: string
    
    OUTPUT
    matrix: matriz nxn
    snt: lista con cada palabra de la frase como elemntos
    '''
    snt = phrase.split()
    n = len(snt)
    matrix =  [[[] for _ in range(n)] for _ in range(n)]
    return snt, matrix

* Paso 3. Implementar en Python el algoritmo CKY probabilístico.

In [5]:
def join_boxes(R, matrix, r_1, c_1, r_2, c_2, r_objetivo, c_objetivo):
    '''
    Se forman las etiquetas y se obtienen sus respectivas probabilidades de las combinaciones entre 2 casillas
    de la matriz y se guardan en la casilla correspondiente.
    
    INPUT:
    R: Diccionario con reglas gramaticales
    matrix: matriz nxn
    r_1: fila de una de las casillas a combinar
    c_1: columna de una de las casillas a combinar
    r_2: fila de la otra casilla a combinar
    c_2: columna de una de las casillas a combinar
    r_objetivo: fila de la casilla donde se guardarán los resultados
    c_objetivo: columna de la casilla donde se guardarán los resultados
    
    OUTPUT:
    matriz actualizada con etiquetas y probabilidades
    '''
    for i in matrix[r_1][c_1]:
        etiqueta_1 = i[0]
        prob_1 = i[1]
            
        for j in matrix[r_2][c_2]:
            etiqueta_2 = j[0]
            prob_2 = j[1]
                
            etiqueta = [etiqueta_1, etiqueta_2]
                
            for rule in R:
                if (etiqueta in R[rule]):
                    prob = prob_1 * prob_2 * R[rule][2]
                    matrix[r_objetivo][c_objetivo].append([R[rule][0],prob, [etiqueta_1,etiqueta_2], [r_1, c_1], [r_2, c_2]])
                    
    return matrix

In [6]:
def CKY(phrase, R):
    '''
    Implementación del algoritmo CKY para obtener matriz con etiquetas y probabilidades. Además de la probabilidad
    la matriz almacena para cada celda la combincación de etiquetas con las que se formó y las coordenas de la casilla.
    
    INPUT:
    phrase: string con la frase a evaluar
    R: diccionario con reglas gramaticales
    
    OUTPUT: 
    matriz con las etiquetas, probabilidades asociadas, etiquetas de origen y casillas de origen.
    '''
    snt, matrix = create_matrix(phrase)
    
    for N in range(len(matrix)):
        r_1 = 0 # row
        c_1 = 0 # column
                
        ## PRIMERA DIAGONAL
        if N == 0:
            for word in snt:
                for rule in R:
                    if(word in R[rule][1]):
                        matrix[r_1][c_1].append([R[rule][0], R[rule][2], [word], [r_1, c_1]])
                r_1+=1
                c_1+=1
        
        ## SEGUNDA DIAGONAL
        if N == 1:
            r_2 = 1
            c_2 = 1
            r_objetivo = 0
            c_objetivo = 1

            for n in range(len(matrix)-1):
                matrix = join_boxes(R, matrix, r_1, c_1, r_2, c_2, r_objetivo, c_objetivo)
        
                r_1 += 1
                c_1 += 1
                r_2 += 1
                c_2 += 1
                r_objetivo += 1
                c_objetivo += 1
            
        ## TERCERA DIAGONAL
        if N == 2:
            r_2 = 1
            c_2 = 2
            r_objetivo = 0
            c_objetivo = 2

            for n in range(len(matrix)-2):
                for m in range(2):                
                    if m != 0:
                        c_1 += 1
                        r_2 += 1     
                    matrix = join_boxes(R, matrix, r_1, c_1, r_2, c_2, r_objetivo, c_objetivo)

                r_1 += 1
                c_2 += 1
                r_objetivo += 1
                c_objetivo += 1
            
        ## CUARTA DIAGONAL
        if N == 3:         
            r_2 = 1
            c_2 = 3
            r_objetivo = 0
            c_objetivo = 3
            
            for n in range(len(matrix)-3):
                for m in range(3):
                    if m != 0:
                        c_1 += 1
                        r_2 += 1  
                    matrix = join_boxes(R, matrix, r_1, c_1, r_2, c_2, r_objetivo, c_objetivo)
            
                r_1 += 1
                c_1 -= 1
                r_2 -= 1
                c_2 += 1
                r_objetivo += 1
                c_objetivo += 1  
            
        ## QUINTA DIAGONAL
        if N == 4:            
            r_2 = 1
            c_2 = 4
            r_objetivo = 0
            c_objetivo = 4
            
            for n in range(len(snt)-4):
                for m in range(4):
                    if m != 0:
                        c_1 += 1
                        r_2 += 1
                    matrix = join_boxes(R, matrix, r_1, c_1, r_2, c_2, r_objetivo, c_objetivo)            
    return matrix    
    

* Paso 4. Aplicar el algoritmo a la frase que se quiere analizar usando la matriz tabla, operando por columnas de abajo arriba y de izquierda a derecha. Para cada celda se deben determinar los constituyentes sintácticos posibles y calcular sus probabilidades.

In [7]:
matriz = CKY(phrase, R)
matriz

[[[['NP', 0.002, ['time'], [0, 0]],
   ['Nominal', 0.002, ['time'], [0, 0]],
   ['VP', 0.004, ['time'], [0, 0]],
   ['Verb', 0.01, ['time'], [0, 0]],
   ['Noun', 0.01, ['time'], [0, 0]]],
  [['S', 1.28e-05, ['NP', 'VP'], [0, 0], [1, 1]],
   ['NP', 8e-07, ['Nominal', 'Nominal'], [0, 0], [1, 1]],
   ['Nominal', 2.0000000000000003e-06, ['Nominal', 'Noun'], [0, 0], [1, 1]],
   ['VP', 6e-06, ['Verb', 'NP'], [0, 0], [1, 1]]],
  [['S', 5.1200000000000005e-09, ['NP', 'VP'], [0, 1], [2, 2]]],
  [],
  [['S', 9.600000000000002e-13, ['NP', 'VP'], [0, 0], [1, 4]],
   ['NP', 2.4000000000000005e-14, ['Nominal', 'Nominal'], [0, 0], [1, 4]],
   ['S', 1.1520000000000003e-13, ['NP', 'VP'], [0, 1], [2, 4]],
   ['Nominal', 6.000000000000002e-14, ['Nominal', 'PP'], [0, 1], [2, 4]]]],
 [[],
  [['NP', 0.002, ['flies'], [1, 1]],
   ['Nominal', 0.002, ['flies'], [1, 1]],
   ['VP', 0.008, ['flies'], [1, 1]],
   ['Verb', 0.02, ['flies'], [1, 1]],
   ['Noun', 0.01, ['flies'], [1, 1]]],
  [['S', 1.28e-05, ['NP', 'V

Como se puede observar, la matriz es simétrica, ya que solo se llena de la diagonal principal hacia arriba. Analicemos una de las casillas para ejemplificar qué representa cada ejemplo.

In [8]:
matriz[0][0]

[['NP', 0.002, ['time'], [0, 0]],
 ['Nominal', 0.002, ['time'], [0, 0]],
 ['VP', 0.004, ['time'], [0, 0]],
 ['Verb', 0.01, ['time'], [0, 0]],
 ['Noun', 0.01, ['time'], [0, 0]]]

En el resultado anterior se muestra la celda correspondiete a la casilla (0,0) es decir la casilla superior izquierda. En primer lugar tenemos las etiquetas formadas, posteriormente, se tiene su probabilidad asociada. El tercer elemento de la lista representa la etiqueta de origen, en este caso solo es 'time'. Finalmente, se tiene la casilla de origen de la etiqueta, pero como en este caso se trata de una casilla en la diagonal principal, solo tiene la casilla (0,0).

Chequemos la esquina superior derecha, con los esultados finales.


In [9]:
matriz[0][4]

[['S', 9.600000000000002e-13, ['NP', 'VP'], [0, 0], [1, 4]],
 ['NP', 2.4000000000000005e-14, ['Nominal', 'Nominal'], [0, 0], [1, 4]],
 ['S', 1.1520000000000003e-13, ['NP', 'VP'], [0, 1], [2, 4]],
 ['Nominal', 6.000000000000002e-14, ['Nominal', 'PP'], [0, 1], [2, 4]]]

Aqui se observa que ya se tienen etiquetas "S", con una probabilidad asociada. De igual forma, se tiene el cómo se forma la etiqueta, es decir a partir de qué otras etiquetas, y las coordenadas de las casillas a las cuales corresponden.

* Paso 5. Implementar el algoritmo para el cálculo de la ruta de Viterbi más probable. Como resultado se debe obtener el árbol de análisis sintáctico más probable.

In [10]:
def definir_ruta_optima(matriz):
    oraciones_completas = []
    arbol = {}

    ## De la casilla superior derecha de la matriz se guardan aquellos registros con etiqueta "S", que representan
    ## una oración completa en la gramática.
    for k in matriz[0][len(matriz)-1]:
        if k[0] == 'S':
            oraciones_completas.append(k)
    
    ## Se encuentra la oración completa con mayor probabilidad asociada.
    probabilidad_mayor = 0
    for i in range(len(oraciones_completas)):
        if oraciones_completas[i][1] > probabilidad_mayor:
            ## Se guarda la mayor probabilidad encontrada
            probabilidad_mayor = oraciones_completas[i][1]
            ## Se guarda el registro con la mayor probabilidad
            ruta_optima = oraciones_completas[i]
            
    return ruta_optima, arbol

In [11]:
def nodo_hijo(ruta_optima):
    nivel = []
    casilla_origen_1 = ruta_optima[3]
    casilla_origen_2 = ruta_optima[4]

    for i in range(2):
        if i == 0:
            casilla_completa = matriz[casilla_origen_1[0]][casilla_origen_1[1]]
        else:
            casilla_completa = matriz[casilla_origen_2[0]][casilla_origen_2[1]]
    
        for elemento in casilla_completa:
            if elemento[0] == ruta_optima[2][i]:
                nivel.append(elemento[2])
    
                if i == 1:
                    ruta_optima = elemento
    
    return nivel, ruta_optima

In [12]:
def show_tree(arbol):
    contador = 0

    for nivel in arbol:
        if contador == 0:
            root = Node(arbol[nivel])
        elif contador == 1:
            level_1_child_1 = Node(arbol[nivel][0], root)
            level_1_child_2 = Node(arbol[nivel][1], root)
        elif contador == 2:
            level_2_child_1 = Node(arbol[nivel][0][0], level_1_child_1)
            level_2_child_2 = Node(arbol[nivel][1][0], level_1_child_2)
            level_2_child_3 = Node(arbol[nivel][1][1], level_1_child_2)
        elif contador == 3:
            level_3_child_1 = Node(arbol[nivel][0][0], level_2_child_2)
            level_3_child_2 = Node(arbol[nivel][1][0], level_2_child_3)
            level_3_child_3 = Node(arbol[nivel][1][1], level_2_child_3)
        elif contador == 4:
            level_4_child_1 = Node(arbol[nivel][0][0], level_3_child_2)
            level_4_child_2 = Node(arbol[nivel][1][0], level_3_child_3)
            level_4_child_3 = Node(arbol[nivel][1][1], level_3_child_3)
        elif contador == 5:
            level_5_child_1 = Node(arbol[nivel][0][0], level_4_child_2)
            level_5_child_2 = Node(arbol[nivel][1][0], level_4_child_3)
        contador+=1

    for pre, fill, node in RenderTree(root):
        print("%s%s" % (pre, node.name))

In [13]:
def construir_arbol(ruta_optima, arbol):
    
    for n in range(6):
        if n == 0:
            key = 'Nivel_' + str(n)
            arbol[key] = ruta_optima[0]
        
        elif n == 1:
            nivel_1 = [ruta_optima[2][0], ruta_optima[2][1]]
            key = 'Nivel_' + str(n)
            arbol[key] = nivel_1
        
        elif n == 2:
            nivel_2, ruta_optima = nodo_hijo(ruta_optima)
            key = 'Nivel_' + str(n)
            arbol[key] = nivel_2
        
        elif n == 3:
            nivel_3, ruta_optima = nodo_hijo(ruta_optima)
            key = 'Nivel_' + str(n)
            arbol[key] = nivel_3
    
        elif n == 4:
            nivel_4, ruta_optima = nodo_hijo(ruta_optima)
            key = 'Nivel_' + str(n)
            arbol[key] = nivel_4
        
        elif n == 5:
            nivel_5 = []
        
            casilla_origen_1 = ruta_optima[3]
            casilla_origen_2 = ruta_optima[4]
        
            for i in range(2):
                if i == 0:
                    casilla_completa = matriz[casilla_origen_1[0]][casilla_origen_1[1]]
                else:
                    casilla_completa = matriz[casilla_origen_2[0]][casilla_origen_2[1]]
            
                for elemento in casilla_completa:                
                    if elemento[0] == ruta_optima[2][i]:
                        nivel_5.append(elemento[2])
                    
            key = 'Nivel_' + str(n)
            arbol[key] = nivel_5
            
    return show_tree(arbol)

Posteriormente, se obtiene la ruta que forma una etiqueta "S" (oración completa) con la probabilidad más alta, qué es la siguinte

In [14]:
ruta_optima, arbol = definir_ruta_optima(matriz)
ruta_optima

['S', 9.600000000000002e-13, ['NP', 'VP'], [0, 0], [1, 4]]

* Paso 6. Proponer una forma de representación del árbol.

Finalmente, se visualiza el árbol

In [15]:
arbol = construir_arbol(ruta_optima, arbol)

S
├── NP
│   └── time
└── VP
    ├── Verb
    │   └── flies
    └── PP
        ├── Preposition
        │   └── like
        └── NP
            ├── Det
            │   └── an
            └── Nominal
                └── arrow
